In [1]:
import time
from create_partitions_by_invariant import create_partitions_by_invariant
from cluster_reaction_centers import cluster_reaction_centers
from invariant_combinations import invariant_combinations
from util import load_reactions
from invariants import own_weisfeiler_leman_convergence_invariant, own_weisfeiler_leman_one_iteration_invariant
import json
import random

In [2]:
from util import load_reactions

reactions = load_reactions(path="C:/Users/Mark/Documents/graph_theory/ITS_largerdataset.pkl.gz")

In [5]:
def wp3(reactions, invariant_config):
    partitions = []
    partition_development = []

    complete_start_time = time.time()

    for invariant_index in range(0, len(invariant_config.values())):
        invariant_start_time = time.time()
        invariant_check_function = invariant_config[invariant_index]

        if invariant_index == 0:
            partitions = create_partitions_by_invariant(reactions, invariant_check_function)
        else:
            all_sub_partitions = []
            for partition in partitions:
                sub_partitions = create_partitions_by_invariant(partition, invariant_check_function)

                all_sub_partitions = all_sub_partitions + sub_partitions

            partitions = all_sub_partitions
        invariant_end_time = time.time()
        print(f"Partitions found {len(partitions)} in {invariant_end_time - invariant_start_time} sec" )



        
        partition_development.append({'number_of_partitions': len(partitions), 'time_elapsed_by_this_invariant': invariant_end_time - invariant_start_time})
    
    pre_filtering_end_time = time.time()
    overall_partitions = []

    for pre_filtered_partition in partitions:
        p = cluster_reaction_centers(pre_filtered_partition)
        overall_partitions = overall_partitions + p 

    complete_end_time = time.time()

    complete_elapsed_time = complete_end_time - complete_start_time
    prefiltering_elapsed_time = pre_filtering_end_time - complete_start_time

    return {
        "invariant_config": [(key, value.__name__) for key, value in invariant_config.items()],
        "partition_development": partition_development,
        "prefiltering_elapsed_time": prefiltering_elapsed_time,
        "complete_elapsed_time": complete_elapsed_time,
        "partitions_found": len(overall_partitions)
    }


In [11]:
from invariants import edge_count_invariant, av_length_invariant, rank_invariant, vertex_count_invariant, labeled_vertex_invariant, vertex_degree_invariant, weisfeiler_leman_one_iteration



invariant_config = {
    0: edge_count_invariant,
    1: av_length_invariant,
    2: own_weisfeiler_leman_one_iteration_invariant
}

result = wp3(reactions, invariant_config)

Partitions found 18 in 319.08966732025146 sec
Partitions found 85 in 453.4054367542267 sec
Partitions found 634 in 1613.7072546482086 sec


In [10]:
result

{'invariant_config': [(0, 'edge_count_invariant'),
  (1, 'labeled_vertex_invariant'),
  (2, 'rank_invariant'),
  (3, 'own_weisfeiler_leman_one_iteration_invariant')],
 'partition_development': [{'number_of_partitions': 18,
   'time_elapsed_by_this_invariant': 327.4632239341736},
  {'number_of_partitions': 606,
   'time_elapsed_by_this_invariant': 855.2205219268799},
  {'number_of_partitions': 626,
   'time_elapsed_by_this_invariant': 792.2290825843811},
  {'number_of_partitions': 634,
   'time_elapsed_by_this_invariant': 203.68345713615417}],
 'prefiltering_elapsed_time': 2178.597285270691,
 'complete_elapsed_time': 3221.4902291297913,
 'partitions_found': 754}